In [ ]:
import plotly.express as px

from core.stats_utils import *
from core.sql_utils import *
from core.pandas_utils import *
from core.plt_utils import *
from core.caching_utils import cache_result
from transform.fleet_info.main import fleet_info
from transform.processed_tss.ProcessedTimeSeries import ProcessedTimeSeries

In [ ]:
fleet_info

In [ ]:
charges = (
    pd.read_parquet("data_cache/tesla_charges.parquet")
    .pipe(left_merge, fleet_info, "vin", "vin", src_dest_cols=["capacity", "tesla_code", "fleet", "fleet_name"])
    .query("soc_diff > 20 & soh.between(0.75, 1.0)")
)

In [ ]:
mean_soh = charges["soh"].mean()
inside_temp_soh_lr = lr_params_as_series(charges, "inside_temp", "soh")
inside_temp_soh_lr

In [ ]:
charges = (
    charges
    .eval("soh_offset_pred = inside_temp * @inside_temp_soh_lr['slope'] + @inside_temp_soh_lr['intercept']")
    .eval("offseted_soh = soh - soh_offset_pred + @mean_soh")
)

px.scatter(
    charges.eval("fleet_name = fleet_name.fillna('Unknown')"),
    x="inside_temp",
    y="offseted_soh",
    trendline="ols",
    trendline_scope="overall",
    color="fleet_name",
)

In [ ]:
soh_per_vin = (
    charges
    .groupby("vin")
    .agg(
        soh=("soh", "median"),
        odometer=("odometer", "last"),
        fleet_name=("fleet_name", "first"),
    )
    .reset_index()
    .pipe(left_merge, fleet_info, "vin", "vin", src_dest_cols=["odometer"])
)
soh_per_vin.query("fleet_name == 'Ayvens'").to_csv("data_cache/raw_soh_per_vin.csv", index=False, float_format="%.2f")

In [ ]:
px.scatter(
    soh_per_vin,
    x="odometer",
    y="soh",
    color="fleet_name",
    trendline="ols",
    trendline_scope="overall",
)